# Bank Marketing Data Set (Save Model)
The data is related with direct marketing campaigns of a Portuguese banking institution.  The marketing campaigns were based on phone calls.  A number of features such as age, kind of job, marital status, education level, credit default, existence of housing loan, etc. were considered.  The classification goal is to predict if the client will subscribe (yes/no) a term deposit.

More information regarding the data set is at https://archive.ics.uci.edu/ml/datasets/bank+marketing#.

<font color=blue>__ _The objective is to show the need for storing a model.  A rudimentary hack to store the model is implemented._ __</font>

## Attribute Information:

### Input variables:
#### Bank client data:
1. age (numeric)
2. job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3. marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4. education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5. default: has credit in default? (categorical: 'no','yes','unknown')
6. housing: has housing loan? (categorical: 'no','yes','unknown')
7. loan: has personal loan? (categorical: 'no','yes','unknown')

#### Related with the last contact of the current campaign:
8. contact: contact communication type (categorical: 'cellular','telephone') 
9. month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10. day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11. duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.

#### Other attributes:
12. campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13. pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14. previous: number of contacts performed before this campaign and for this client (numeric)
15. poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')

#### Social and economic context attributes:
16. emp.var.rate: employment variation rate - quarterly indicator (numeric)
17. cons.price.idx: consumer price index - monthly indicator (numeric) 
18. cons.conf.idx: consumer confidence index - monthly indicator (numeric) 
19. euribor3m: euribor 3 month rate - daily indicator (numeric)
20. nr.employed: number of employees - quarterly indicator (numeric)

### Output variable (desired target):
21. y - has the client subscribed a term deposit? (binary: 'yes','no')

## Import packages

In [1]:
from hana_ml import dataframe
from hana_ml.algorithms.pal import linear_model
from hana_ml.algorithms.pal import clustering
import numpy as np
import matplotlib.pyplot as plt
import logging

## Setup logging

In [2]:
logging.basicConfig()
logger = logging.getLogger('hana_ml.ml_base')
logger.setLevel(logging.INFO)
logger.addHandler(logging.NullHandler())

## Setup connection and data sets
The data is loaded into 4 tables - full set, test set, training set, and the validation set:
<li>DBM2_RFULL_TBL</li>
<li>DBM2_RTEST_TBL</li>
<li>DBM2_RTRAINING_TBL</li>
<li>DBM2_RVALIDATION_TBL</li>

To do that, a connection is created and passed to the loader.

There is a config file, <b>config/e2edata.ini</b> that controls the connection parameters and whether or not to reload the data from scratch.  In case the data is already loaded, there would be no need to load the data.  A sample section is below.  If the config parameter, reload_data is true then the tables for test, training, and validation are (re-)created and data inserted into them.

#########################<br>
[hana]<br>
url=host.sjc.sap.corp<br>
user=username<br>
passwd=userpassword<br>
port=3xx15<br>
<br>

#########################<br>

In [3]:
from data_load_utils import DataSets, Settings
url, port, user, pwd = Settings.load_config("../../config/e2edata.ini")
connection_context = dataframe.ConnectionContext(url, port, user, pwd)
full_tbl, training_tbl, validation_tbl, test_tbl = DataSets.load_bank_data(connection_context)
training_set = connection_context.table(training_tbl)
validation_set = connection_context.table(validation_tbl)
features = ['AGE','JOB','MARITAL','EDUCATION','DBM_DEFAULT', 'HOUSING','LOAN','CONTACT','DBM_MONTH','DAY_OF_WEEK','DURATION','CAMPAIGN','PDAYS','PREVIOUS','POUTCOME','EMP_VAR_RATE','CONS_PRICE_IDX','CONS_CONF_IDX','EURIBOR3M','NREMPLOYED']
label = "LABEL"

# Create model
Use a specific value for the hyper parameters to keep it simple.  In this case the hyperparameters are lamb and alpha corresponding to enet_lambda and enet_alpha.

In [4]:
lr = linear_model.LogisticRegression(connection_context, solver='Cyclical', tol=0.000001, max_iter=10000, stat_inf=True,
                                       pmml_export='multi-row', lamb=0.000, alpha=0.010, 
                                       class_map0='no', class_map1='yes')

lr.fit(training_set, features=features, label=label)

accuracy_val = lr.score(validation_set, 'ID', features=features, label=label)
print('Accuracy=%f' %(accuracy_val))

# Model has already been persisted.
#model_persistence.save(connection_context, lr, 'nk_lr')

Accuracy=0.915013


## Model Analysis
For simplicity, just look at coefficients to see if they are all meaningful.  In general, one may want to do PCA and graph the data to see what features should be used so that the model generalizes well.  In addition, see which features affect the outcome.

Note the statement below.  The absolute value of the coefficient is calculated and sorted on this absolute value.  The method collect() is what brings the data to the client, otherwise all the computation is on the server and the data (actually just a reference to the data set as defined by a SQL statement) remains there.

In [5]:
print(lr.coef_.select('*', ('abs("COEFFICIENT")', 'ABS_COEFFICIENT')).sort(['ABS_COEFFICIENT'], desc=True).collect())

                                  VARIABLE_NAME  COEFFICIENT  ABS_COEFFICIENT
0                             __PAL_INTERCEPT__   -53.920002        53.920002
1                   DBM_MONTH__PAL_DELIMIT__mar     1.616431         1.616431
2                POUTCOME__PAL_DELIMIT__success     1.056200         1.056200
3                                  EMP_VAR_RATE    -1.053383         1.053383
4                                CONS_PRICE_IDX     1.016365         1.016365
5            EDUCATION__PAL_DELIMIT__illiterate     0.950523         0.950523
6                   DBM_MONTH__PAL_DELIMIT__may    -0.688407         0.688407
7               CONTACT__PAL_DELIMIT__telephone    -0.634011         0.634011
8                   DBM_MONTH__PAL_DELIMIT__sep    -0.590638         0.590638
9                                     EURIBOR3M     0.509001         0.509001
10                  DBM_MONTH__PAL_DELIMIT__nov    -0.498424         0.498424
11           POUTCOME__PAL_DELIMIT__nonexistent     0.490490    

# Save Model
What we want to be able to do now is to save the model using a save command on the LogisticRegression object.
Below is a hack that saves the PMML model to a specific table.  In general, we would want to save many attributes including the __coefficients__ so they can be compared to other models that are saved.

In [6]:
print(lr.result_.select_statement)
with connection_context.connection.cursor() as cur:
    try:
        cur.execute('DROP TABLE {0}'.format("MYMODEL"))
    except:
        pass
lr.result_.save("MYMODEL")   # save("scenario-name", "description", "model-name")
#  SAVE MODEL ....  This is not the desired API and is only for illustration
# model.add_attribute("paramters", (enet_alpha,...))
# model.add_attribute("data set used", ...)
# model.add_attribute("tested by", "nanda")
# model.add_attribute("coefficients", lr.coef_)

SELECT * FROM "#LR_RESULT_TBL_0"


In [7]:
df = connection_context.table("MYMODEL")
print(df.select_statement)

SELECT * FROM "MYMODEL"
